# Processamento das Entidades 

Nesse notebook, os arquivos gerados pelo BERT são processados para a geração dos arquivos finais com as entidades. O processamento consiste em eliminar algumas entidades que não ajudam muito no entendimento do texto, como: 'Presidente', 'Ministro', 'Sr', 'Sra'. O objetivo é deixar em evidência entidades que tragam mais informação para a análise dos textos processados, como por exemplo: Presidente da República, Senador Randolfe e assim por diante. 

Essa eliminação foi feita de forma manual, selecionando algumas entidades para cada uma das classes extraídas. 

Outro processamento realizado foi a unificação de termos como COVID e COVID-19. 

Por fim, as entidades das classes `COISA`, `ABSTRACCAO`, `OUTRO`, `OBRA` e `ACONTECIMENTO` foram agrupadas em uma única entidade: `OUTROS`. 

In [9]:
import pandas as pd 
import json

Palavras removidas por entidades

In [2]:
remove_words = {'PESSOA': [
    'Sr. Presidente',
    'Sr. Relator',
    'Prefeitos',
    'Presidente',
    'Senador',
    'Senadora',
    'Ministro',
    'Governador',
    'Senhor',
    'Excelência',
    'Relator',
    'V. Exa.',
    'V. Exa',
    'V. Exas',
    'Governador do',
    'Srs.',
    'Sr.',
    'Sra',
    'Sras',
    'Sra.',
    'Sras.',
    'Sr',
    'Srs',
    'Secretário de',
    'Covid-19',
    'Governador',
    'Governadores',
    'Prefeitos',
    'Sras. Senadoras',
    'Senadores', 'Senadoras', 'Sr. Senador', 'Srs. Senadores', 'Deputado', 'Secretário', 'Secretários',
    'Parlamentar', 'Parlamentares', 'Deputados', ],
    'LOCAL': ['Território', 'Brasil', 'País', 'Município', 'Municípios', 'Estado', 'Estados', 'Países', 'Covid-19'],
    "ORGANIZACAO": ['Município', 'Governo', 'Municípios', 'Estado', 'Estados', 'Países', 'País', 'Brasil', 'Covid-19', 'China'],
    "OUTROS": ['Covid-19', 'País', 'Programa', 'Medicina']

}


Mapeamento para unificar algunas alguns termos:

In [3]:
map_words = {
    'Covid': 'Covid-19',
    'Programa Nacional de Imunizações': 'PNI'
}

Classes que foram agrupadas em Outros.

In [4]:
class_to_outros = ['COISA','ABSTRACCAO','OUTRO','OBRA','ACONTECIMENTO']

O código a seguir acessa cada arquivo gerado pelo modelo de Reconhecimento de Entidades nomeadas e agrupa em duas variáveis: `all_entites` que tem todas as entidades independente do dia e `all_entities_date` que agrupa por data.

In [15]:
import os 

path_ = "../output/ner_depoentes_convidados/ner_files/"

all_files = os.listdir(path_)

all_entites_by_date = {}
all_entites = {}
all_entities_by_date_name = {}


for file_ in all_files:
    file_split = file_.split("_")
    date_ = file_split[1]
    name_ = ' '.join(file_split[2:]).split(".")[0]
    
    with open(path_ + file_) as datafile:
        data_ = json.load(datafile)

    final_dict = {}
    for data in data_[0]['entities']:
        text_ = data['text']
        text_ = text_.strip()

        if text_ in map_words.keys():
            text_ = map_words[text_]

        class_ = data['class']

        if class_ in class_to_outros:
            class_ = 'OUTROS'
        
        if class_ in remove_words.keys():
            words_to_remove = remove_words[class_]
        else:
            words_to_remove = []

        if text_ not in words_to_remove:



            if class_ not in final_dict.keys():
                final_dict[class_] = {text_: 1}
            else: 
                if text_ not in final_dict[class_].keys():
                    final_dict[class_][text_] = 1
                else:
                    final_dict[class_][text_] += 1

            if class_ not in all_entites.keys():
                all_entites[class_] = {text_: 1}
            else:
                if text_ not in all_entites[class_].keys():
                    all_entites[class_][text_] = 1
                else:
                    all_entites[class_][text_] += 1

            if date_ not in all_entites_by_date.keys():
                all_entites_by_date[date_] = {}

            if class_ not in all_entites_by_date[date_].keys():
                all_entites_by_date[date_][class_] = {text_: 1}
            else: 
                if text_ not in all_entites_by_date[date_][class_].keys():
                    all_entites_by_date[date_][class_][text_] = 1
                else:
                    all_entites_by_date[date_][class_][text_] += 1


    for class_ in final_dict.keys():
        final_dict[class_] = dict(sorted(final_dict[class_].items(), key=lambda item: item[1], reverse=True))

    if date_ not in all_entities_by_date_name.keys():
        all_entities_by_date_name[date_] = [{name_: final_dict}]
    else:
        all_entities_by_date_name[date_].append({name_: final_dict})

    for class_ in all_entites.keys():
        all_entites[class_] = dict(sorted(all_entites[class_].items(), key=lambda item: item[1], reverse=True))

    for date_ in all_entites_by_date.keys():
        for class_ in all_entites_by_date[date_].keys():
            all_entites_by_date[date_][class_] = dict(sorted(all_entites_by_date[date_][class_].items(), key=lambda item: item[1], reverse=True))
        

In [41]:
all_data = []
for date_ in all_entities_by_date_name.keys():
    for entry in all_entities_by_date_name[date_]:
        line_ = [(date_, key, entitie_type, word, entry[key][entitie_type][word]) for key in entry.keys() for entitie_type in entry[key].keys() for word in entry[key][entitie_type].keys()]
        all_data.extend(line_)

df_data = pd.DataFrame(all_data, columns=['data','depoente_convidado','tipo_entidade','entidade','frequencia'])
df_data.sort_values(by=['data','depoente_convidado'],inplace=True)
df_data.reset_index(drop=True,inplace=True)
df_data

,data,depoente_convidado,tipo_entidade,entidade,frequencia
0,20210504,LUIZ HENRIQUE MANDETTA,ORGANIZACAO,Ministério da Saúde,74
1,20210504,LUIZ HENRIQUE MANDETTA,ORGANIZACAO,SUS,21
2,20210504,LUIZ HENRIQUE MANDETTA,ORGANIZACAO,Organização Mundial da Saúde,18
3,20210504,LUIZ HENRIQUE MANDETTA,ORGANIZACAO,Câmara,12
4,20210504,LUIZ HENRIQUE MANDETTA,ORGANIZACAO,Fiocruz,10
...,...,...,...,...,...
9711,20210715,CRISTIANO ALBERTO HOSSRI CARVALHO,VALOR,200 milhões,1
9712,20210715,CRISTIANO ALBERTO HOSSRI CARVALHO,VALOR,em torno de 20 anos,1
9713,20210715,CRISTIANO ALBERTO HOSSRI CARVALHO,VALOR,123 Milhas,1
9714,20210715,FÁBIO HENRIQUE MING MARTINI,PESSOA,Fábio Ming,1


Os dicionários criados são salvos em três arquivos na pasta `output`: 

* `all_entities`: foi salvo em `todas_entidades_nomeadas.json`.
* `all_entities_by_date`: foi salvo em `todas_entidades_nomeadas_por_data.json`.
* `all_entities_by_date_name`: foi salvo em `tordas_entidades_nomeadas_por_data_nome.csv`.

In [6]:
with open("../output/todas_entidades_nomeadas.json", 'w') as outfile:
    json.dump(all_entites, outfile)

In [7]:
with open("../output/todas_entidades_nomeadas_por_data.json", 'w') as outfile:
    json.dump(all_entites_by_date, outfile)

In [42]:
df_data.to_csv("../output/todas_entidades_nomeadas_por_data_nome.csv",index=False)